In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df_train = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

print(df_train.head())
print(df_test.head())

Initial Exploratory Data Analysis (EDA) on df_train to understand its structure and contents

In [ ]:
print(df_train.info())

In [ ]:
print(df_train.head())

In [ ]:
df_train.describe()

In [ ]:
print(df_train.isnull().sum())

In [ ]:
cols_nos = df_train[['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']]

In [ ]:
mean_time_spent_alone = df_train['Time_spent_Alone'].mean()

In [ ]:
df_train['Time_spent_Alone'] = df_train['Time_spent_Alone'].fillna(mean_time_spent_alone)
print(df_train['Time_spent_Alone'].info())

In [ ]:
print(df_train.info())

In [ ]:
imputation_means = {}
for col in cols_nos:
    imputation_means[col] = df_train[col].mean()
    df_train[col] = df_train[col].fillna(imputation_means[col])

In [ ]:
print("Calculated Means for Imputation:")
print(imputation_means)

print("\ndf_train.info() after numerical imputation:")
print(df_train.info())

In [ ]:
cols_cat = df_train[['Stage_fear', 'Drained_after_socializing']]

In [ ]:
(df_train['Stage_fear'].mode())

In [ ]:
imputation_modes = {}
for col in cols_cat:
    imputation_modes[col] = df_train[col].mode()[0]
    df_train[col] = df_train[col].fillna(imputation_modes[col])

In [ ]:
print("Calculated Modes for Imputation:")
print(imputation_modes)

print("\ndf_train.info() after categorical imputation:")
print(df_train.info())

In [ ]:
cols_to_clean_string = ['Stage_fear', 'Drained_after_socializing']

for col in cols_to_clean_string:
    df_train[col] = df_train[col].astype(str).str.strip().str.lower()

print("--- After String Hygiene ---")
for col in cols_to_clean_string:
    print(f"\nColumn: {col}")
    print(df_train[col].unique())
    print(df_train[col].value_counts())

In [ ]:
mapping = {'no': 0, 'yes': 1}
df_train[cols_to_clean_string] = df_train[cols_to_clean_string].replace(mapping)
print(df_train[cols_to_clean_string].head())

In [ ]:
print(df_train.info())

In [ ]:
df_train['Personality'].unique()

In [ ]:
df_train['Personality'].astype(str).str.strip().str.lower()
mapping = {'Extrovert': 1, 'Introvert': 0}
df_train['Personality'] = df_train['Personality'].replace(mapping)

In [ ]:
df_train['Personality'].unique()
print(df_train.info())

In [ ]:
df_train.columns

In [ ]:
# X = df_train[['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
#        'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
#        'Post_frequency']]
X = df_train.drop(columns = ['id', 'Personality'])
y = df_train['Personality']

In [ ]:
print(X.columns)

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
model = LogisticRegression(random_state=16)

In [ ]:
model.fit(X, y)

In [ ]:
df_test.head()

df_test.info()

df_test.isnull().sum()

In [ ]:
cols_nos = df_test[['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']]

In [ ]:
for col in cols_nos:
    mean_from_train = imputation_means[col] 
    df_test[col] = df_test[col].fillna(mean_from_train)

In [ ]:
print(df_test.info())

In [ ]:
cols_cat = df_train[['Stage_fear', 'Drained_after_socializing']]

In [ ]:
for col in cols_cat:
    df_test[col] = df_test[col].fillna(imputation_modes[col])

In [ ]:
print(df_test.info())

In [ ]:
cols_to_clean_string = ['Stage_fear', 'Drained_after_socializing']

for col in cols_to_clean_string:
    df_test[col] = df_test[col].astype(str).str.strip().str.lower()

In [ ]:
print("--- After String Hygiene ---")
for col in cols_to_clean_string:
    print(f"\nColumn: {col}")
    print(df_test[col].unique())
    print(df_test[col].value_counts())

In [ ]:
df_test[cols_to_clean_string].head()

In [ ]:
mapping = {'no': 0, 'yes': 1}
df_test[cols_to_clean_string] = df_test[cols_to_clean_string].replace(mapping)
print(df_test[cols_to_clean_string].head())

In [ ]:
df_test[cols_to_clean_string].head()

In [ ]:
print(df_test.info())

In [ ]:
X_test = df_test[X.columns]

In [ ]:
test_predictions_numeric = model.predict(X_test)

print("Predictions made successfully! Here are the first 5 predictions (numeric):")
print(test_predictions_numeric[:5])

In [ ]:
print(test_predictions_numeric)

In [ ]:
reverse_map = {0: 'Introvert', 1: "Extrovert"}

In [ ]:
s = pd.Series(test_predictions_numeric).map(reverse_map)

In [ ]:
print(s)

In [ ]:
submission_df = pd.DataFrame({
    'id': df_test['id'],          
    'Personality': s              
})

In [ ]:
print(submission_df.head())

In [ ]:
submission_df.to_csv('submission.csv', index=False)